<font size="+3"><strong>6.3. Clustering with Multiple Features</strong></font>

In the previous lesson, we built a K-Means model to create clusters of respondents to the Survey of Consumer Finances. We made our clusters by looking at two features only, but there are hundreds of features in the dataset that we didn't take into account and that could contain valuable information. In this lesson, we'll examine all the features, selecting five to create clusters with. After we build our model and choose an appropriate number of clusters, we'll learn how to visualize multi-dimensional clusters in a 2D scatter plot using something called principal component analysis (PCA). 

In [ ]:

import pandas as pd
import plotly.express as px
import wqet_grader
from IPython.display import VimeoVideo
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

wqet_grader.init("Project 6 Assessment")


In [ ]:
VimeoVideo("714612789", h="f4f8c10683", width=600)

# Prepare Data

## Import

We spent some time in the last lesson zooming in on a useful subset of the SCF, and this time, we're going to zoom in even further. One of the persistent issues we've had with this dataset is that it includes some outliers in the form of ultra-wealthy households. This didn't make much of a difference for our last analysis, but it could pose a problem in this lesson, so we're going to focus on families with net worth under \\$2 million.

In [ ]:
VimeoVideo("714612746", h="07dc57f72c", width=600)

**Task 6.3.1:** Rewrite your `wrangle` function from the last lesson so that it returns a DataFrame of households whose net worth is less than \\$2 million and that have been turned down for credit or feared being denied credit in the past 5 years (see `"TURNFEAR"`). 

- [<span id='technique'>Write a function in <span id='tool'>Python</span></span>.](../%40textbook/02-python-advanced.ipynb#Functions)
- [Subset a DataFrame by selecting one or more columns in pandas.](../%40textbook/04-pandas-advanced.ipynb#Subsetting-with-Masks)

In [ ]:
def wrangle(filepath):
    
    return df

In [ ]:
df = ...
print(df.shape)
df.head()

## Explore

In this lesson, we want to make clusters using more than two features, but which of the 351 features should we choose? Often times, this decision will be made for you. For example, a stakeholder could give you a list of the features that are most important to them. If you don't have that limitation, though, another way to choose the best features for clustering is to determine which numerical features have the largest **variance**. That's what we'll do here. 

In [ ]:
VimeoVideo("714612679", h="040facf6e2", width=600)

**Task 6.3.2:** Calculate the variance for all the features in `df`, and create a Series `top_ten_var` with the 10 features with the largest variance.

- [What's variance?](../%40textbook/05-pandas-summary-statistics.ipynb#Variance)
- [Calculate the variance of a DataFrame or Series in pandas.](../%40textbook/05-pandas-summary-statistics.ipynb#Variance)

In [ ]:
# Calculate variance, get 10 largest features
top_ten_var = ...
top_ten_var

As usual, it's harder to make sense of a list like this than it would be if we visualized it, so let's make a graph. 

In [ ]:
VimeoVideo("714612647", h="5ecf36a0db", width=600)

**Task 6.3.3:** Use plotly express to create a horizontal bar chart of `top_ten_var`. Be sure to label your x-axis `"Variance"`, the y-axis `"Feature"`, and use the title `"SCF: High Variance Features"`.

- [What's a bar chart?](../%40textbook/07-visualization-pandas.ipynb#Bar-Charts)
- [Create a bar chart using plotly express.](../%40textbook/08-visualization-plotly.ipynb#Bar-Chart)

In [ ]:
# Create horizontal bar chart of `top_ten_var`
fig = ...

fig.show()

One thing that we've seen throughout this project is that many of the wealth indicators are highly skewed, with a few outlier households having enormous wealth. Those outliers can affect our measure of variance. Let's see if that's the case with one of the features from `top_five_var`.

In [ ]:
VimeoVideo("714612615", h="9ae23890fc", width=600)

**Task 6.3.4:** Use plotly express to create a horizontal boxplot of `"NHNFIN"` to determine if the values are skewed. Be sure to label the x-axis `"Value [$]"`, and use the title `"Distribution of Non-home, Non-Financial Assets"`.

- [What's a boxplot?](../%40textbook/06-visualization-matplotlib.ipynb#Boxplots)
- [Create a boxplot using plotly express.](../%40textbook/08-visualization-plotly.ipynb#Boxplots)

In [ ]:
# Create a boxplot of `NHNFIN`
fig = ...

fig.show()

Whoa! The dataset is massively right-skewed because of the huge outliers on the right side of the distribution. Even though we already excluded households with a high net worth with our `wrangle` function, the variance is still being distorted by some extreme outliers.

The best way to deal with this is to look at the **trimmed variance**, where we remove extreme values before calculating variance. We can do this using the `trimmed_variance` function from the `SciPy` library.

In [ ]:
VimeoVideo("714612570", h="b1be8fb750", width=600)

**Task 6.3.5:** Calculate the trimmed variance for the features in `df`. Your calculations should not include the top and bottom 10% of observations. Then create a Series `top_ten_trim_var` with the 10 features with the largest variance.

- [What's trimmed variance?](../%40textbook/05-pandas-summary-statistics.ipynb#Variance)
- [Calculate the trimmed variance of data using SciPy.](../%40textbook/05-pandas-summary-statistics.ipynb#Variance)
- [Apply a function to a DataFrame in pandas.](../%40textbook/04-pandas-advanced.ipynb#Applying-Functions-to-DataFrames-and-Series)

In [ ]:
# Calculate trimmed variance
top_ten_trim_var = ...
top_ten_trim_var

Okay! Now that we've got a better set of numbers, let's make another bar graph.

In [ ]:
VimeoVideo("714611188", h="d762a98b1e", width=600)

**Task 6.3.6:** Use plotly express to create a horizontal bar chart of `top_ten_trim_var`. Be sure to label your x-axis `"Trimmed Variance"`, the y-axis `"Feature"`, and use the title `"SCF: High Variance Features"`.

- [What's a bar chart?](../%40textbook/07-visualization-pandas.ipynb#Bar-Charts)
- [Create a bar chart using plotly express.](../%40textbook/08-visualization-plotly.ipynb#Bar-Chart)

In [ ]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = ...

fig.show()

There are three things to notice in this plot. First, the variances have decreased a lot. In our previous chart, the x-axis went up to \\$80 billion; this one goes up to \\$12 billion. Second, the top 10 features have changed a bit. All the features relating to business ownership (`"...BUS"`) are gone. Finally, we can see that there are big differences in variance from feature to feature. For example, the variance for `"WAGEINC"` is around than \\$500 million, while the variance for `"ASSET"` is nearly \\$12 billion. In other words, these features have completely different scales. This is something that we'll need to address before we can make good clusters. 

In [ ]:
VimeoVideo("714611161", h="61dee490ee", width=600)

**Task 6.3.7:** Generate a list `high_var_cols` with the column names of the  five features with the highest trimmed variance.

- [What's an index?](../%40textbook/01-python-getting-started.ipynb#Working-with-Lists)
- [Access the index of a DataFrame or Series in pandas.](../%40textbook/03-pandas-getting-started.ipynb#Working-with-DataFrame-Indices)

In [ ]:
high_var_cols = ...
high_var_cols

## Split

Now that we've gotten our data to a place where we can use it, we can follow the steps we've used before to build a model, starting with a feature matrix.

In [ ]:
VimeoVideo("714611148", h="f7fbd4bcc5", width=600)

**Task 6.3.8:** Create the feature matrix `X`. It should contain the five columns in `high_var_cols`.

- [What's a feature matrix?](../%40textbook/14-ml-classification.ipynb#Randomized-Train-Test-split)
- [Subset a DataFrame by selecting one or more columns in pandas.](../%40textbook/04-pandas-advanced.ipynb#Subset-a-DataFrame-by-Selecting-One-or-More-Columns) 

In [ ]:
X = ...
print("X shape:", X.shape)
X.head()

# Build Model

## Iterate

During our EDA, we saw that we had a scale issue among our features. That issue can make it harder to cluster the data, so we'll need to fix that to help our analysis along. One strategy we can use is **standardization**, a statistical method for putting all the variables in a dataset on the same scale. Let's explore how that works here. Later, we'll incorporate it into our model pipeline. 

In [ ]:
VimeoVideo("714611113", h="3671a603b5", width=600)

**Task 6.3.9:** Create a DataFrame `X_summary` with the mean and standard deviation for all the features in `X`.

- [Aggregate data in a DataFrame using one or more functions in pandas.](../%40textbook/03-pandas-getting-started.ipynb#Working-with-DataFrames)

In [ ]:
X_summary = ...
X_summary

That's the information we need to standardize our data, so let's make it happen.

In [ ]:
VimeoVideo("714611056", h="670f6bdb78", width=600)

**Task 6.3.10:** Create a `StandardScaler` transformer, use it to transform the data in `X`, and then put the transformed data into a DataFrame named `X_scaled`.

- [What's standardization?](../%40textbook/13-ml-data-pre-processing-and-production.ipynb#Standardization)
- [Transform data using a transformer in scikit-learn.](../%40textbook/13-ml-data-pre-processing-and-production.ipynb#One-Hot-Encoding)

In [ ]:
# Instantiate transformer
ss = ...

# Transform `X`
X_scaled_data = ...

# Put `X_scaled_data` into DataFrame
X_scaled = ...

print("X_scaled shape:", X_scaled.shape)
X_scaled.head()

As you can see, all five of the features use the same scale now. But just to make sure, let's take a look at their mean and standard deviation.

In [ ]:
VimeoVideo("714611032", h="1ed03c46eb", width=600)

**Task 6.3.11:** Create a DataFrame `X_scaled_summary` with the mean and standard deviation for all the features in `X_scaled`.

- [Aggregate data in a DataFrame using one or more functions in pandas.](../%40textbook/03-pandas-getting-started.ipynb#Working-with-DataFrames) 

In [ ]:
X_scaled_summary = ...
X_scaled_summary

And that's what it should look like. Remember, standardization takes all the features and scales them so that they all have a mean of 0 and a standard deviation of 1.

Now that we can compare all our data on the same scale, we can start making clusters. Just like we did last time, we need to figure out how many clusters we should have.

In [ ]:
VimeoVideo("714610976", h="82f32af967", width=600)

**Task 6.3.12:** Use a `for` loop to build and train a K-Means model where `n_clusters` ranges from 2 to 12 (inclusive). Your model should include a `StandardScaler`. Each time a model is trained, calculate the inertia and add it to the list `inertia_errors`, then calculate the silhouette score and add it to the list `silhouette_scores`.

- [Write a `for` loop in Python.](../%40textbook/01-python-getting-started.ipynb#Working-with-for-Loops)
- [Calculate the inertia for a model in scikit-learn.](../%40textbook/16-ml-unsupervised-learning.ipynb#Silhouette-Score)
- [Calculate the silhouette score for a model in scikit-learn.](../%40textbook/16-ml-unsupervised-learning.ipynb#Silhouette-Score)
- [Create a pipeline in scikit-learn.](../%40textbook/13-ml-data-pre-processing-and-production.ipynb#Creating-a-Pipeline-in-scikit-learn)

In [ ]:
n_clusters = ...
inertia_errors = ...
silhouette_scores = ...

# Add `for` loop to train model and calculate inertia, silhouette score.


print("Inertia:", inertia_errors[:3])
print()
print("Silhouette Scores:", silhouette_scores[:3])

Just like last time, let's create an elbow plot to see how many clusters we should use. 

In [ ]:
VimeoVideo("714610940", h="bacf42a282", width=600)

**Task 6.3.13:** Use plotly express to create a line plot that shows the values of `inertia_errors` as a function of `n_clusters`. Be sure to label your x-axis `"Number of Clusters"`, your y-axis `"Inertia"`, and use the title `"K-Means Model: Inertia vs Number of Clusters"`.

- [What's a line plot?](../%40textbook/07-visualization-pandas.ipynb#Line-Plots)
- [Create a line plot in plotly express.](../%40textbook/07-visualization-pandas.ipynb#Line-Plots)

In [ ]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = ...

fig.show()

You can see that the line starts to flatten out around 4 or 5 clusters.

<div class="alert alert-block alert-info">
    <p><b>Note:</b> We ended up using 5 clusters last time, too, but that's because we're working with very similar data. 5 clusters isn't always going to be the right choice for this type of analysis, as we'll see below.</p></div>

Let's make another line plot based on the silhouette scores.

In [ ]:
VimeoVideo("714610912", h="01961ee57a", width=600)

**Task 6.3.14:** Use plotly express to create a line plot that shows the values of `silhouette_scores` as a function of `n_clusters`. Be sure to label your x-axis `"Number of Clusters"`, your y-axis `"Silhouette Score"`, and use the title `"K-Means Model: Silhouette Score vs Number of Clusters"`.

- [What's a line plot?](../%40textbook/07-visualization-pandas.ipynb#Line-Plots)
- [Create a line plot in plotly express.](../%40textbook/07-visualization-pandas.ipynb#Line-Plots)

In [ ]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = ...
fig.update_layout(
    xaxis_title="Number of Clusters", yaxis_title="Silhouette Score"

fig.show()

This one's a little less straightforward, but we can see that the best silhouette scores occur when there are 3 or 4 clusters. 

Putting the information from this plot together with our inertia plot, it seems like the best setting for `n_clusters` will be 4. 

In [ ]:
VimeoVideo("714610883", h="a6a0431b02", width=600)

**Task 6.3.15:** Build and train a new k-means model named `final_model`. Use the information you gained from the two plots above to set an appropriate value for the `n_clusters` argument. Once you've built and trained your model, submit it to the grader for evaluation.

- [Create a pipeline in scikit-learn.](../%40textbook/13-ml-data-pre-processing-and-production.ipynb#Creating-a-Pipeline-in-scikit-learn)
- [Fit a model to training data in scikit-learn.](../%40textbook/13-ml-data-pre-processing-and-production.ipynb#Creating-a-Pipeline-in-scikit-learn)

In [ ]:
final_model = ...


When you're confident in your model, submit it to the grader.

In [ ]:

wqet_grader.grade("Project 6 Assessment", "Task 6.3.14", final_model)

# Communicate

It's time to let everyone know how things turned out. Let's start by grabbing the labels.

In [ ]:
VimeoVideo("714610862", h="69ff3fb2c8", width=600)

**Task 6.3.16:** Extract the labels that your `final_model` created during training and assign them to the variable `labels`.

- [Access an object in a pipeline in scikit-learn.](../%40textbook/13-ml-data-pre-processing-and-production.ipynb#Accessing-an-Object-in-a-Pipeline)

In [ ]:
labels = ...
print(labels[:5])

We're going to make a visualization, so we need to create a new DataFrame to work with.

In [ ]:
VimeoVideo("714610842", h="008a463aca", width=600)

**Task 6.3.17:** Create a DataFrame `xgb` that contains the mean values of the features in `X` for each of the clusters in your `final_model`.

- [Access an object in a pipeline in scikit-learn.](../%40textbook/13-ml-data-pre-processing-and-production.ipynb#Accessing-an-Object-in-a-Pipeline)
- [Aggregate data using the groupby method in pandas.](../%40textbook/04-pandas-advanced.ipynb#Series-and-Groupby)
- [Create a DataFrame from a Series in pandas.](../%40textbook/04-pandas-advanced.ipynb#Series-and-Groupby)

In [ ]:
xgb = ...
xgb

Now that we have a DataFrame, let's make a bar chart and see how our clusters differ. 

In [ ]:
VimeoVideo("714610772", h="e118407ff1", width=600)

**Task 6.3.18:** Use plotly express to create a side-by-side bar chart from `xgb` that shows the mean of the features in `X` for each of the clusters in your `final_model`. Be sure to label the x-axis `"Cluster"`, the y-axis `"Value [$]"`, and use the title `"Mean Household Finances by Cluster"`.

- [What's a bar chart?](../%40textbook/07-visualization-pandas.ipynb#Bar-Charts)
- [Create a bar chart using plotly express.](../%40textbook/08-visualization-plotly.ipynb#Bar-Chart)

In [ ]:
# Create side-by-side bar chart of `xgb`
fig = ...

fig.show()

Remember that our clusters are based partially on `NETWORTH`, which means that the households in the 0 cluster have the smallest net worth, and the households in the 2 cluster have the highest. Based on that, there are some interesting things to unpack here.

First, take a look at the `DEBT` variable. You might think that it would scale as net worth increases, but it doesn't. The lowest amount of debt is carried by the households in cluster 2, even though the value of their houses (shown in green) is roughly the same. You can't *really* tell from this data what's going on, but one possibility might be that the people in cluster 2 have enough money to pay down their debts, but not quite enough money to leverage what they have into additional debts. The people in cluster 3, by contrast, might not need to worry about carrying debt because their net worth is so high. 

Finally, since we started out this project looking at home values, take a look at the relationship between `DEBT` and `HOUSES`. The value of the debt for the people in cluster 0 is higher than the value of their houses, suggesting that most of the debt being carried by those people is tied up in their mortgages — if they own a home at all. Contrast that with the other three clusters: the value of everyone else's debt is lower than the value of their homes.

So all that's pretty interesting, but it's different from what we did last time, right? At this point in the last lesson, we  made a scatter plot. This was a straightforward task because we only worked with two features, so we could plot the data points in two dimensions. But now `X` has five dimensions! How can we plot this to give stakeholders a sense of our clusters?

Since we're working with a computer screen, we don't have much of a choice about the number of dimensions we can use: it's got to be two. So, if we're going to do anything like the scatter plot we made before, we'll need to take our 5-dimensional data and change it into something we can look at in 2 dimensions.

In [ ]:
VimeoVideo("714610665", h="19c9f7bf7f", width=600)

**Task 6.3.19:** Create a `PCA` transformer, use it to reduce the dimensionality of the data in `X` to 2, and then put the transformed data into a DataFrame named `X_pca`. The columns of `X_pca` should be named `"PC1"` and `"PC2"`.

- [What's principal component analysis (PCA)?](../%40textbook/16-ml-unsupervised-learning.ipynb#Principal-Component-Analysis)
- [Transform data using a transformer in scikit-learn.](../%40textbook/13-ml-data-pre-processing-and-production.ipynb#One-Hot-Encoding)

In [ ]:
# Instantiate transformer
pca = ...

# Transform `X`
X_t = ...

# Put `X_t` into DataFrame
X_pca = ...

print("X_pca shape:", X_pca.shape)
X_pca.head()

So there we go: our five dimensions have been reduced to two. Let's make a scatter plot and see what we get.

In [ ]:
VimeoVideo("714610491", h="755c66fe15", width=600)

**Task 6.3.20:** Use plotly express to create a scatter plot of `X_pca` using seaborn. Be sure to color the data points using the labels generated by your `final_model`. Label the x-axis `"PC1"`, the y-axis `"PC2"`, and use the title `"PCA Representation of Clusters"`.

- [What's a scatter plot?](../%40textbook/06-visualization-matplotlib.ipynb#Scatter-Plots)
- [Create a scatter plot using plotly express.](../%40textbook/08-visualization-plotly.ipynb#Scatter-Plots)

In [ ]:
# Create scatter plot of `PC2` vs `PC1`
fig = ...

fig.show()

<div class="alert alert-block alert-info">
<b>Note:</b> You can often improve the performance of PCA by <a href="https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html">standardizing your data first</a>. Give it a try by including a <code>StandardScaler</code> in your transformation of <code>X</code>. How does it change the clusters in your scatter plot? 
</div>

One limitation of this plot is that it's hard to explain what the axes here represent. In fact, both of them are a combination of the five features we originally had in `X`, which means this is pretty abstract. Still, it's the best way we have to show as much information as possible as an explanatory tool for people outside the data science community. 

So what does this graph mean? It means that we made four tightly-grouped clusters that share some key features. If we were presenting this to a group of stakeholders, it might be useful to show this graph first as a kind of warm-up, since most people understand how a two-dimensional object works. Then we could move on to a more nuanced analysis of the data.

Just something to keep in mind as you continue your data science journey.

---
Copyright © 2022 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
